# Stellar Evolution
## Introduction to Astronomy, AGS Winterim 2023
By: Mathilda Avirett-Mackenzie

We've learned about the life cycles of stars: how they are born in [molecular gas clouds](https://esahubble.org/images/heic0601a/), spend most of their lifetimes fusing hydrogen into helium, then expand into red giants as they fuse heavier elements, until they run out of fuel and either [peacefully shed their outer layers](https://esahubble.org/images/heic1310a/), leaving behind a white dwarf (low mass stars) or [explode in dramatic supernovae](https://esahubble.org/images/heic0515a/), collapsing into a neutron star or black hole (high mass stars), leaving behind diffuse gas that may eventually form into more stars. 

We've also learned how these stages of a star's life appear to us on Earth and how to tell a star's fate by its position on the Hertzsprung-Russell diagram. We've even [played around](https://starinabox.lco.global/#) with how stars move along the diagram when they run out of hydrogen, ignite helium and heavier elements, and eventually run out of fuel completely.

In this notebook, we'll learn how to use the [astropy](https://www.astropy.org) and [matplotlib](https://matplotlib.org) packages to read in a table of stars and their magnitudes and colors, then plot these in an HR diagram.

In [ ]:
# packages we will use
import numpy as np               # needed for some calculations
import matplotlib.pyplot as plt  # does plotting
from astropy.table import Table  # to read in data tables

## Stellar data
We'll be using magnitudes, parallaxes, and colors of stars measured by the [Hipparcos satellite](https://www.esa.int/Science_Exploration/Space_Science/Hipparcos_overview) in the early 1990s. Hipparcos was the first space telescope devoted to performing astrometry, precisely measuring the positions of millions of stars and their movement within the Milky Way. Its successor, [Gaia](https://www.esa.int/Science_Exploration/Space_Science/Gaia), is currently operating and recently published its third data release, which has increased the number of stars with these measurements to several billion, three orders of magnitude more!

These data can be accessed via the [VizieR](http://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=I/239/hip_main) website.

**_Fun fact:_ HIPPARCOS is an acronym for HIgh Precision PARallax COllecting Satellite. Astronomers love dumb acronyms.**

In [ ]:
hip_cat = Table.read('../data/hipparcos.fit')            # read in the catalogue to a Table
hip_cat = hip_cat[hip_cat['Plx'] > 3*hip_cat['e_Plx']*3] # removing stars with unreliable parallax measurements

In [ ]:
hip_cat # we can look at what we have in the table

Within this catalogue, the columns we care about are the **magnitude** `Vmag`, the **parallax** `Plx`, and the **color** `B-V`. 

The **absolute magnitude** $M$ of a star is defined as its magnitude seen at a distance of 10 parsecs. It is related to the apparent magnitude $m$ of the star seen at its distance $d$ from Earth by
$$m - M = 5\log(d) - 5.$$
Solving for $M$ gives us
$$M = m  - 5\log(d) + 5$$
and plugging in the relationship between distance and parallax, or $d = 1/p$, we get that
$$M = m + 5\log(p) + 5$$

In [ ]:
m_abs = hip_cat['Vmag'] + 5*np.log10(hip_cat['Plx']/1000) + 5 # note that parallax is in milliarcseconds

We can also convert to luminosities with the information that the Sun has an absolute magnitude of $4.83$. From the definitions of flux and magnitude (see the intro notebook for a reminder), we have that
$$M = 4.83 - 2.5\log(L)$$
or solving for $L$,
$$L = 10^{0.4(4.83 - M)}$$

In [ ]:
lumin = 10**(0.4*(4.83 - m_abs))

We can read color straight out of the catalogue, though it may seem a bit odd to express color as a number, or a difference of two numbers. This is because in astronomy, the word color means the flux ratio of red light to blue light coming from a star. Expressed in magnitude space, this becomes a difference.

In [ ]:
color = hip_cat['B-V']

## Hertzsprung-Russell Diagram
We can now plot the luminosities against the colors and see whether everything looks right!

In [ ]:
fig = plt.figure(figsize=(20,20))   # create a figure
ax = fig.subplots(nrows=1, ncols=1) # create the plot on the figure

ax.scatter(color, lumin, c='k', s=1, zorder=1) # scatter plot of the colors and luminosities

# labelling axes
ax.set_xlabel(r'$B-V$ color', size=24)
ax.set_ylabel(r'Luminosity ($L_{\odot}$)', size=24)

# formatting stuff, don't bother reading this bit
ax.set_yscale('log')
ax.grid('True', zorder=-1) 
ax.set_xlim(-0.5, 2)
ax.set_ylim(1e-3, 1e3)
ax.tick_params(which='minor', left=True, right=True, direction='in')
ax.tick_params(which='major', left=True, right=True, bottom=True, top=True, direction='in', labelsize=20)